In [23]:
!pip install biom-format

In [326]:
!pip install -U kaleido

     |████████████████████████████████| 79.9 MB 259 kB/s eta 0:00:01


In [89]:
from biom import load_table
input_biom = "../Human_Subramanian/INPUT_FILES/Subramanian_et_al_otu_table/Subramanian_et_al_otu_table.biom"
table = load_table(input_biom)

In [90]:
df_abundances = table.to_dataframe().T.rename_axis('sampleID').astype(np.int32).reset_index()
print(df_abundances.shape)
df_abundances.head()

(1897, 79598)


,sampleID,304309,16076,373909,35786,591907,410908,117676,326792,189116,...,GGS.ReferenceOTU193,GGS.ReferenceOTU191,GGS.ReferenceOTU190,GGS.ReferenceOTU197,GGS.ReferenceOTU195,GGS.ReferenceOTU199,GGS.ReferenceOTU294,GGS.ReferenceOTU66,GGS.ReferenceOTU61,GGS.ReferenceOTU229
0,Bgsng7114.m22,1,0,0,0,0,0,0,507,0,...,0,0,0,0,0,0,0,0,0,0
1,Bgsng8064.m14,11,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Bgsng7082.m24,0,1369,0,0,0,0,0,81,0,...,0,0,0,0,0,0,0,0,0,0
3,Bgsng7192.m22,0,47,0,0,0,0,0,39,0,...,0,0,0,0,0,0,0,0,0,0
4,Bgsng7082.m19,0,48,0,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,0


In [91]:
df_abundances["304309"]

0        1
1       11
2        0
3        0
4        0
        ..
1892     0
1893     0
1894     0
1895     0
1896     0
Name: 304309, Length: 1897, dtype: Sparse[int32, 0]

In [92]:
file_name = "../Human_Subramanian/INPUT_FILES/Subramanian_et_al_mapping_file.txt"
df_metadata = pd.read_csv(file_name, sep=None, engine='python')

In [93]:
df_metadata.columns

Index(['#SampleID', 'BarcodeSequence', 'LinkerPrimerSequence', 'PersonID',
       'FamilyID', 'ENA.SampleID', 'ENA.LibraryName', 'Health_Analysis_Groups',
       'tax_id', 'scientific_name', 'common_name', 'anonymized_name',
       'sample_title', 'sample_description', 'investigation type',
       'project name', 'target gene', 'sequencing method', 'collection date',
       'host subject id', 'age_in_months', 'sex', 'environmental package',
       'geographic location_latitude', 'geographic location_longitude',
       'geographic location_country', 'environment_biome',
       'environment_feature', 'environment_material', 'Description'],
      dtype='object')

In [94]:
# TODO: handle group and reference_group

In [95]:
df_metadata.age_in_months *= 30

In [96]:

# id_columns = ["FamilyID", "ENA.SampleID", "tax_id"]


In [97]:
df_metadata = df_metadata.rename({"#SampleID": "sampleID", 
               "PersonID": "subjectID",
               "age_in_months": "age_at_collection",}, axis=1)   

other_names = {}
for col1 in set(df_metadata.columns).difference(set(MicrobiomeDataset.specific_columns)):
    other_names[col1] = f"meta_{col1}"
    
df_metadata = df_metadata.rename(other_names, axis=1)        
    

In [98]:
columns_to_remove = ["ENA.SampleID", 
                     "anonymized_name",
                     "sample_title",
                     "host subject id",
                     "Description"]
for col in df_metadata.columns:
    if len(df_metadata[col].unique()) == 1:
        print(col, "has only 1 unique value", df_metadata[col].unique())
        df_metadata.drop(col, inplace=True, axis=1)
    elif col in columns_to_remove:
        df_metadata.drop(col, inplace=True, axis=1)

meta_LinkerPrimerSequence has only 1 unique value [nan]
meta_tax_id has only 1 unique value [408170]
meta_scientific_name has only 1 unique value ['human gut metagenome']
meta_common_name has only 1 unique value ['human gut metagenome']
meta_sample_description has only 1 unique value ['human gut metagenome']
meta_investigation type has only 1 unique value ['mimarks-survey']
meta_project name has only 1 unique value ['Subramanian maturity']
meta_target gene has only 1 unique value ['V4']
meta_sequencing method has only 1 unique value ['Illumina MiSeq']
meta_collection date has only 1 unique value ['None']
meta_environmental package has only 1 unique value ['human-gut']
meta_environment_biome has only 1 unique value ['ENVO:human-associated habitat']
meta_environment_feature has only 1 unique value ['ENVO:human-associated habitat']
meta_environment_material has only 1 unique value ['ENVO:feces']


In [99]:
df_metadata.columns

Index(['sampleID', 'meta_BarcodeSequence', 'subjectID', 'meta_FamilyID',
       'meta_ENA.SampleID', 'meta_ENA.LibraryName',
       'meta_Health_Analysis_Groups', 'meta_anonymized_name',
       'meta_sample_title', 'meta_host subject id', 'age_at_collection',
       'meta_sex', 'meta_geographic location_latitude',
       'meta_geographic location_longitude',
       'meta_geographic location_country', 'meta_Description'],
      dtype='object')

In [100]:
df_metadata.shape

(1897, 16)

In [101]:
df = pd.merge(df_abundances, df_metadata, on='sampleID')

In [102]:
df.shape

(1894, 79613)

In [103]:
df.head()

,sampleID,304309,16076,373909,35786,591907,410908,117676,326792,189116,...,meta_Health_Analysis_Groups,meta_anonymized_name,meta_sample_title,meta_host subject id,age_at_collection,meta_sex,meta_geographic location_latitude,meta_geographic location_longitude,meta_geographic location_country,meta_Description
0,Bgsng7114.m22,1,0,0,0,0,0,0,507,0,...,Healthy Singletons,Bgsng7114,Bgsng7114.m22,Bgsng7114,630.0,male,23.933,88.983,Bangladesh,Bgsng7114.m22
1,Bgsng8064.m14,11,0,0,0,0,0,0,1,0,...,Healthy Singletons,Bgsng8064,Bgsng8064.m14,Bgsng8064,393.0,male,23.933,88.983,Bangladesh,Bgsng8064.m14
2,Bgsng7082.m24,0,1369,0,0,0,0,0,81,0,...,Healthy Singletons,Bgsng7082,Bgsng7082.m24,Bgsng7082,714.0,female,23.933,88.983,Bangladesh,Bgsng7082.m24
3,Bgsng7192.m22,0,47,0,0,0,0,0,39,0,...,Healthy Singletons,Bgsng7192,Bgsng7192.m22,Bgsng7192,636.0,female,23.933,88.983,Bangladesh,Bgsng7192.m22
4,Bgsng7082.m19,0,48,0,0,0,0,0,4,0,...,Healthy Singletons and MAM cross-sectional ana...,Bgsng7082,Bgsng7082.m19,Bgsng7082,543.0,female,23.933,88.983,Bangladesh,Bgsng7082.m19


In [104]:
df.dtypes

sampleID                                        object
304309                                Sparse[int64, 0]
16076                                 Sparse[int64, 0]
373909                                Sparse[int64, 0]
35786                                 Sparse[int64, 0]
                                            ...       
meta_sex                                        object
meta_geographic location_latitude              float64
meta_geographic location_longitude             float64
meta_geographic location_country                object
meta_Description                                object
Length: 79613, dtype: object

In [105]:
df.to_csv("../Human_Subramanian/INPUT_FILES/subramanian_et_al_data.csv", index=False)

KeyboardInterrupt: 

---

In [113]:
!ls ../Human_Subramanian/INPUT_FILES/

RAW_DATA		  subramanian_et_al_l4.csv  subramanian_et_al_l7.csv
subramanian_et_al_l2.csv  subramanian_et_al_l5.csv
subramanian_et_al_l3.csv  subramanian_et_al_l6.csv


In [200]:
file_name = "../Human_Subramanian/INPUT_FILES/subramanian_et_al_l4.csv"
df_metadata = pd.read_csv(file_name, sep=None, engine='python')

In [201]:
df_metadata.columns.tolist()

['Unnamed: 0',
 'x.sampleid',
 'barcodesequence',
 'linkerprimersequence',
 'personid',
 'familyid',
 'ena.sampleid',
 'ena.libraryname',
 'health_analysis_groups',
 'tax_id',
 'scientific_name',
 'common_name',
 'anonymized_name',
 'sample_title',
 'sample_description',
 'investigation.type',
 'project.name',
 'target.gene',
 'sequencing.method',
 'collection.date',
 'host.subject.id',
 'age_in_months',
 'sex',
 'environmental.package',
 'geographic.location_latitude',
 'geographic.location_longitude',
 'geographic.location_country',
 'environment_biome',
 'environment_feature',
 'environment_material',
 'description',
 'unassigned.other',
 'k__archaea.p__crenarchaeota',
 'k__archaea.p__euryarchaeota',
 'k__bacteria.other',
 'k__bacteria.p__ad3',
 'k__bacteria.p__actinobacteria',
 'k__bacteria.p__bacteroidetes',
 'k__bacteria.p__chlamydiae',
 'k__bacteria.p__cyanobacteria',
 'k__bacteria.p__elusimicrobia',
 'k__bacteria.p__firmicutes',
 'k__bacteria.p__fusobacteria',
 'k__bacteria.p__

In [202]:
df_metadata["ena.libraryname"].unique()

array(['BANG_HLTHY', 'HS1_Pool6', 'HS1_Pool5', 'HS1_Pool7', 'HS1_Pool4',
       'ACMAL_3_SS2  and  ACMAL_3_SS4', 'SS3', 'SS1', 'Twins34',
       'Twins12', 'Twins56_run1 and Twins56_run2', 'Twins78',
       'TwinsCP2_run1 and TwinsCP2_run2', 'TwinsRX',
       'ACMAL_1_run1 and ACMAL_1_run2', 'ACMAL_2_run1 and ACMAL_2_run2',
       'GGS89', 'GGS7', 'GGS6', 'GGS3'], dtype=object)

In [203]:
# TODO: handle group and reference_group

In [204]:
df_metadata.age_in_months *= 30

In [205]:

# id_columns = ["FamilyID", "ENA.SampleID", "tax_id"]


In [206]:
df_metadata = df_metadata.rename(
    {"x.sampleid": "sampleID", 
   "personid": "subjectID",
   "age_in_months": "age_at_collection",
#     "reference_group": 
     "health_analysis_groups": "group"
    }, axis=1)   

       
    

In [207]:
columns_to_remove = ["Unnamed: 0",
                     "ena.sampleid", 
                     "anonymized_name",
                     "sample_title",
                     "host subject id",
                     "description"]
for col in df_metadata.columns:
    if len(df_metadata[col].unique()) == 1:
        print(col, "has only 1 unique value", df_metadata[col].unique())
        df_metadata.drop(col, inplace=True, axis=1)
    elif col in columns_to_remove:
        df_metadata.drop(col, inplace=True, axis=1)

linkerprimersequence has only 1 unique value [nan]
tax_id has only 1 unique value [408170]
scientific_name has only 1 unique value ['human gut metagenome']
common_name has only 1 unique value ['human gut metagenome']
sample_description has only 1 unique value ['human gut metagenome']
investigation.type has only 1 unique value ['mimarks-survey']
project.name has only 1 unique value ['Subramanian maturity']
target.gene has only 1 unique value ['V4']
sequencing.method has only 1 unique value ['Illumina MiSeq']
collection.date has only 1 unique value ['None']
environmental.package has only 1 unique value ['human-gut']
environment_biome has only 1 unique value ['ENVO:human-associated habitat']
environment_feature has only 1 unique value ['ENVO:human-associated habitat']
environment_material has only 1 unique value ['ENVO:feces']
study has only 1 unique value ['RM_Bangladesh']


In [208]:
other_names = {}
for col1 in set(df_metadata.columns).difference(set(MicrobiomeDataset.specific_columns)):
    if "bacteria" in col1:
        if not col1.startswith("bacteria_"):
            other_names[col1] = f"bacteria_{col1}"
    else:
        other_names[col1] = f"meta_{col1}"
    
df_metadata = df_metadata.rename(other_names, axis=1) 

In [209]:
df_metadata.columns

Index(['sampleID', 'meta_barcodesequence', 'subjectID', 'meta_familyid',
       'meta_ena.libraryname', 'group', 'meta_host.subject.id',
       'age_at_collection', 'meta_sex', 'meta_geographic.location_latitude',
       ...
       'bacteria_k__bacteria.p__tenericutes.c__mollicutes.o__',
       'bacteria_k__bacteria.p__tenericutes.c__mollicutes.o__anaeroplasmatales',
       'bacteria_k__bacteria.p__tenericutes.c__mollicutes.o__mycoplasmatales',
       'bacteria_k__bacteria.p__tenericutes.c__mollicutes.o__rf39',
       'bacteria_k__bacteria.p__tenericutes.c__rf3.o__ml615j.28',
       'bacteria_k__bacteria.p__verrucomicrobia.c__opitutae.o__.cerasicoccales.',
       'bacteria_k__bacteria.p__verrucomicrobia.c__verruco.5.o__wchb1.41',
       'bacteria_k__bacteria.p__verrucomicrobia.c__verrucomicrobiae.o__verrucomicrobiales',
       'bacteria_k__bacteria.p__wps.2.c__.o__',
       'bacteria_k__bacteria.p__.thermi..c__deinococci.o__deinococcales'],
      dtype='object', length=153)

In [218]:
df_metadata.columns

Index(['sampleID', 'meta_barcodesequence', 'subjectID', 'meta_familyid',
       'meta_ena.libraryname', 'group', 'meta_host.subject.id',
       'age_at_collection', 'meta_sex', 'meta_geographic.location_latitude',
       ...
       'bacteria_k__bacteria.p__tenericutes.c__mollicutes.o__',
       'bacteria_k__bacteria.p__tenericutes.c__mollicutes.o__anaeroplasmatales',
       'bacteria_k__bacteria.p__tenericutes.c__mollicutes.o__mycoplasmatales',
       'bacteria_k__bacteria.p__tenericutes.c__mollicutes.o__rf39',
       'bacteria_k__bacteria.p__tenericutes.c__rf3.o__ml615j.28',
       'bacteria_k__bacteria.p__verrucomicrobia.c__opitutae.o__.cerasicoccales.',
       'bacteria_k__bacteria.p__verrucomicrobia.c__verruco.5.o__wchb1.41',
       'bacteria_k__bacteria.p__verrucomicrobia.c__verrucomicrobiae.o__verrucomicrobiales',
       'bacteria_k__bacteria.p__wps.2.c__.o__',
       'bacteria_k__bacteria.p__.thermi..c__deinococci.o__deinococcales'],
      dtype='object', length=153)

In [211]:
df_metadata.head()

,sampleID,meta_barcodesequence,subjectID,meta_familyid,meta_ena.libraryname,group,meta_host.subject.id,age_at_collection,meta_sex,meta_geographic.location_latitude,...,bacteria_k__bacteria.p__tenericutes.c__mollicutes.o__,bacteria_k__bacteria.p__tenericutes.c__mollicutes.o__anaeroplasmatales,bacteria_k__bacteria.p__tenericutes.c__mollicutes.o__mycoplasmatales,bacteria_k__bacteria.p__tenericutes.c__mollicutes.o__rf39,bacteria_k__bacteria.p__tenericutes.c__rf3.o__ml615j.28,bacteria_k__bacteria.p__verrucomicrobia.c__opitutae.o__.cerasicoccales.,bacteria_k__bacteria.p__verrucomicrobia.c__verruco.5.o__wchb1.41,bacteria_k__bacteria.p__verrucomicrobia.c__verrucomicrobiae.o__verrucomicrobiales,bacteria_k__bacteria.p__wps.2.c__.o__,bacteria_k__bacteria.p__.thermi..c__deinococci.o__deinococcales
0,Bgsng7035.m1,GGATAGCCACTTC,Bgsng7035,Bgsng7035,BANG_HLTHY,Healthy Singletons,Bgsng7035,6.0,male,23.933,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Bgsng7035.m2,TTGGGCGTGAACT,Bgsng7035,Bgsng7035,BANG_HLTHY,Healthy Singletons,Bgsng7035,36.0,male,23.933,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bgsng7035.m3,TAACAAGGAACGC,Bgsng7035,Bgsng7035,BANG_HLTHY,Healthy Singletons,Bgsng7035,57.0,male,23.933,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bgsng7035.m4,TCTGCAGTTGGAC,Bgsng7035,Bgsng7035,BANG_HLTHY,Healthy Singletons,Bgsng7035,87.0,male,23.933,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bgsng7035.m5,CTCGGCAATTATG,Bgsng7035,Bgsng7035,BANG_HLTHY,Healthy Singletons,Bgsng7035,108.0,male,23.933,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [212]:
df_metadata.to_csv("../Human_Subramanian/INPUT_FILES/subramanian_et_al_l2_website.csv", index=False)